In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np

In [ ]:
MAX_LEN = 50          # max tokens per complaint
EMBED_DIM = 300       # FastText dimension
NUM_CLASSES = 3
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 5

In [ ]:
class FastTextEncoder:
    def __init__(self, model):
        self.model = model  # gensim fasttext model

    def encode_tokens(self, tokens):
        vectors = []
        for token in tokens:
            if token in self.model:
                vectors.append(self.model[token])
            else:
                vectors.append(np.zeros(EMBED_DIM))
        return np.array(vectors)

In [ ]:
def pad_sequence(embedding_matrix, max_len=MAX_LEN):
    length = embedding_matrix.shape[0]

    if length >= max_len:
        return embedding_matrix[:max_len]

    padding = np.zeros((max_len - length, EMBED_DIM))
    return np.vstack([embedding_matrix, padding])

In [ ]:
class UrgencyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, encoder):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.encoder = encoder

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]

        tokens = self.tokenizer(text)   # your tokeniser
        embeddings = self.encoder.encode_tokens(tokens)
        padded = pad_sequence(embeddings)

        x = torch.tensor(padded, dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)

        return x, y

In [ ]:
class UrgencyModel(nn.Module):
    def __init__(self):
        super(UrgencyModel, self).__init__()

        # Parallel Conv1D layers
        self.conv2 = nn.Conv1d(EMBED_DIM, 150, kernel_size=2, padding=1)
        self.conv3 = nn.Conv1d(EMBED_DIM, 150, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(EMBED_DIM, 150, kernel_size=4, padding=2)

        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=450,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, NUM_CLASSES)

    def forward(self, x):
        # x shape: (B, L, D)

        x = x.permute(0, 2, 1)  # (B, D, L)

        c2 = F.relu(self.conv2(x))
        c3 = F.relu(self.conv3(x))
        c4 = F.relu(self.conv4(x))

        # Trim to same length
        min_len = min(c2.shape[2], c3.shape[2], c4.shape[2])
        c2 = c2[:, :, :min_len]
        c3 = c3[:, :, :min_len]
        c4 = c4[:, :, :min_len]

        x = torch.cat([c2, c3, c4], dim=1)  # (B, 450, L)

        x = x.permute(0, 2, 1)  # (B, L, 450)

        lstm_out, _ = self.lstm(x)

        # Take last time step
        x = lstm_out[:, -1, :]  # (B, 256)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return F.softmax(x, dim=1)

In [ ]:
model = UrgencyModel()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(model, dataloader):
    model.train()

    for epoch in range(EPOCHS):
        total_loss = 0

        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()

            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

In [ ]:
texts = [
    "[caps on] i cannot believe this [multiple exclamations] where is my order [impatient questioning]",
    "order delayed but not urgent"
]

labels = [2, 1]  # Example: 0=Low, 1=Medium, 2=High

# tokenizer = your_tokenizer_function
# fasttext_model = load_fasttext_model()
# encoder = FastTextEncoder(fasttext_model)

# dataset = UrgencyDataset(texts, labels, tokenizer, encoder)
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# train(model, dataloader)